# Facial Reconstruction from Low-Quality CCTV Footage

## Introduction
This notebook aims to develop a machine learning solution for reconstructing human faces from low-quality CCTV footage. The solution will address issues like motion blur, poor lighting, and low resolution to help investigators identify suspects more accurately.

## Objectives for Round 1
1. **Basic Facial Reconstruction Model:** Develop a model to reconstruct or enhance facial images from low-quality CCTV footage.
2. **Image Enhancement Techniques:** Apply methods like super-resolution, noise reduction, or deblurring.
3. **Comparison of Results:** Provide side-by-side comparisons of original and enhanced images.

## Dataset Overview
The dataset consists of .npz files containing video frames, bounding boxes, and landmarks.

### Example Files:
- **Aaron_Eckhart_0.npz**
  - `colorImages.npy (231,237,3,79)`
  - `boundingBox.npy (4,2,79)`
  - `landmarks2D.npy (68,2,79)`
  - `landmarks3D.npy (68,3,79)`

## Step 1: Data Loading and Preprocessing
First, we will load the data and preprocess it for training.




In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from tqdm import tqdm  # Import tqdm for progress tracking
from concurrent.futures import ThreadPoolExecutor



In [2]:

# Define the target shape for images
TARGET_SHAPE = (64, 64, 3)

def resize_image(image):
    return resize(image, TARGET_SHAPE, preserve_range=True, anti_aliasing=True)

def load_npz_file(file_path):
    data = np.load(file_path, mmap_mode='r')
    color_images = data['colorImages'] / 255.0  # Normalize images
    color_images_resized = np.array([resize_image(img) for img in color_images.transpose(3, 0, 1, 2)])
    
    bounding_boxes = data['boundingBox']
    landmarks_2d = data['landmarks2D']
    landmarks_3d = data['landmarks3D']
    
    return color_images_resized, bounding_boxes, landmarks_2d, landmarks_3d

# Load all files in the dataset directory
dataset_dir = 'dataset/'


In [3]:
file_list = [os.path.join(dataset_dir, f) for f in os.listdir(dataset_dir) if f.endswith('.npz')]

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

def data_generator(file_list, batch_size=8):
    while True:
        for i in range(0, len(file_list), batch_size):
            batch_files = file_list[i:i + batch_size]
            images = []
            
            with ThreadPoolExecutor() as executor:
                results = list(tqdm(executor.map(load_npz_file, batch_files), total=len(batch_files), desc="Loading batch"))
            
            for result in results:
                color_images, _, _, _ = result
                images.append(color_images)
            
            # Free memory by discarding the original arrays after concatenation
            images = np.concatenate(images, axis=0)
            augmented_images = np.array([datagen.random_transform(img) for img in images])
            
            yield augmented_images, augmented_images



In [4]:
# Split file list into training and validation sets
train_files, val_files = train_test_split(file_list, test_size=0.2, random_state=42)



## Step 2: Model Development
We will develop a Convolutional Neural Network (CNN) for facial reconstruction.



In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, MaxPooling2D,Dropout, concatenate


In [7]:
def build_unet_model(input_shape):
    inputs = Input(input_shape)
    
    # Encoder
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)  # reduced filters from 64 to 32
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)  # reduced filters from 128 to 64
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    
    # Bottleneck
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)  # reduced filters from 256 to 128
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    
    # Decoder
    up4 = UpSampling2D((2, 2))(conv3)
    concat4 = concatenate([up4, conv2])
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(concat4)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)
    
    up5 = UpSampling2D((2, 2))(conv4)
    concat5 = concatenate([up5, conv1])
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(concat5)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)
    
    outputs = Conv2D(3, (1, 1), activation='sigmoid', padding='same')(conv5)
    
    model = Model(inputs, outputs)
    return model


input_shape = TARGET_SHAPE
model = build_unet_model(input_shape)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 64, 64,    │        896 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 64, 64,    │      9,248 │ conv2d[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 32, 32,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 32, 32,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 32, 32,    │     36,928 │ conv2d_2[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 16, 16,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 16, 16,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 16, 16,    │    147,584 │ conv2d_4[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d       │ (None, 32, 32,    │          0 │ conv2d_5[0][0]    │
│ (UpSampling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32, 32,    │          0 │ up_sampling2d[0]… │
│ (Concatenate)       │ 192)              │            │ conv2d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 32, 32,    │    110,656 │ concatenate[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 32, 32,    │     36,928 │ conv2d_6[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_1     │ (None, 64, 64,    │          0 │ conv2d_7[0][0]    │
│ (UpSampling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 64, 64,    │          0 │ up_sampling2d_1[… │
│ (Concatenate)       │ 96)               │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 64, 64,    │     27,680 │ concatenate_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 64, 64,    │      9,248 │ conv2d_8[0][0]  

 Total params: 471,619 (1.80 MB)

 Trainable params: 471,619 (1.80 MB)

 Non-trainable params: 0 (0.00 B)



## Step 3: Training the Model
We will train the model using the training data and validate it using the validation data.



In [8]:
# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss'),
    tf.keras.callbacks.ModelCheckpoint('unet_model.keras', save_best_only=True, save_freq='epoch')
]

# Train model
train_gen = data_generator(train_files, batch_size=8)
val_gen = data_generator(val_files, batch_size=8)

history = model.fit(
    train_gen,
    epochs=50,
    steps_per_epoch=len(train_files) // 8,
    validation_data=val_gen,
    validation_steps=len(val_files) // 8,
    callbacks=callbacks
)


Loading batch: 100%|██████████| 8/8 [00:04<00:00,  1.90it/s]


Epoch 1/50
  1/219 ━━━━━━━━━━━━━━━━━━━━ 4:57:48 82s/step - accuracy: 0.0940 - loss: 0.0829

Loading batch: 100%|██████████| 8/8 [00:24<00:00,  3.08s/it]


  2/219 ━━━━━━━━━━━━━━━━━━━━ 4:19:34 72s/step - accuracy: 0.2544 - loss: 0.0809

Loading batch: 100%|██████████| 8/8 [00:10<00:00,  1.30s/it]


...

Loading batch: 100%|██████████| 8/8 [00:50<00:00,  6.28s/it]


  4/219 ━━━━━━━━━━━━━━━━━━━━ 5:49:26 98s/step - accuracy: 0.3755 - loss: 0.0802

Loading batch: 100%|██████████| 8/8 [00:23<00:00,  2.94s/it]


  5/219 ━━━━━━━━━━━━━━━━━━━━ 5:27:16 92s/step - accuracy: 0.4079 - loss: 0.0799

Loading batch: 100%|██████████| 8/8 [00:06<00:00,  1.21it/s]


## Step 4: Evaluation and Comparison
We will evaluate the model and compare the original and enhanced images.



In [ ]:
# Load the best model
model.load_weights('unet_model.keras')

# Evaluate the model
val_gen = data_generator(val_files, batch_size=8)
val_loss, val_accuracy = model.evaluate(val_gen, steps=len(val_files) // 8)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')


In [ ]:
def display_images(original, enhanced, n=5):
    plt.figure(figsize=(20, 10))
    for i in range(n):
        # Original images
        plt.subplot(2, n, i + 1)
        plt.imshow(original[i])
        plt.title('Original')
        plt.axis('off')

        # Enhanced images
        plt.subplot(2, n, i + 1 + n)
        plt.imshow(enhanced[i])
        plt.title('Enhanced')
        plt.axis('off')
    plt.show()

# Predict enhanced images
val_gen = data_generator(val_files, batch_size=32)
original_images, _ = next(val_gen)
enhanced_images = model.predict(original_images)

# Display images
display_images(original_images, enhanced_images)



*`This notebook provides a comprehensive approach to developing a facial reconstruction model from scratch, including data loading, preprocessing, model development, training, and evaluation. It also includes optimizations and debugging techniques to handle the large dataset efficiently.`*

